# Case study: Mobilities between a group of countries's and others 

This case investigate mobilities between a group of countries and others. The group in this study case has several large developed economies: Canada, France, Germany, Italy, Japan, the United Kingdom and the United States.

## Import packages

In [3]:
import os
import math
import pandas as pd
import numpy as np
import requests
from io import StringIO
import keplergl
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib as mpl
from keplergl import KeplerGl

Prepare boundary data.

In [15]:
# country group
G = ['USA', 'GBR', "FRA", "ITA", "JPN", "CAN", "DEU"]

# load country boundaries and generate centroids
country_file = r'gdam00_simplified/gadm36_00.shp'
gdf = gpd.read_file(country_file)
gdf['y'] = gdf.centroid.y
gdf['x'] = gdf.centroid.x
gdf

C:\ProgramData\Anaconda3\envs\gdal_env\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  import sys
C:\ProgramData\Anaconda3\envs\gdal_env\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  


,GID_0,NAME_0,geometry,y,x
0,ABW,Aruba,"POLYGON ((-69.97820 12.46986, -70.02847 12.503...",12.509652,-69.970508
1,AFG,Afghanistan,"POLYGON ((68.52644 31.75435, 68.58202 31.75034...",33.828566,66.029628
2,AGO,Angola,"MULTIPOLYGON (((11.73347 -16.67255, 11.74014 -...",-12.297561,17.548715
3,AIA,Anguilla,"MULTIPOLYGON (((-63.42375 18.58903, -63.42403 ...",18.221736,-63.056916
4,ALA,Ã land,"MULTIPOLYGON (((21.32195 59.74986, 21.32472 59...",60.204199,20.147058
...,...,...,...,...,...
251,XSP,Spratly Islands,"MULTIPOLYGON (((114.35472 11.45139, 114.35167 ...",10.816933,114.508408
252,YEM,Yemen,"MULTIPOLYGON (((53.26931 12.12875, 53.28680 12...",15.904209,47.582688
253,ZAF,South Africa,"MULTIPOLYGON (((19.42097 -34.67903, 19.42208 -...",-28.996440,25.086365
254,ZMB,Zambia,"POLYGON ((25.88753 -17.98407, 25.86519 -17.975...",-13.462328,27.792294


Prepare flow data

In [145]:
dict_countries = {}
max_cnt_countries = []

min_cnt = math.inf
max_cnt = -math.inf  

for target_place in G:
    params = {
        "operation": "get_flow_by_place",
        "source": "twitter",
        "scale": "world_country",
        "place": str(target_place),
        "begin": "01/01/2019",
        "end": "12/31/2019",
        "direction": "both"
    }

    q = r'http://gis.cas.sc.edu/GeoAnalytics/REST'
    r = requests.get(q, params=params)
    df = pd.read_csv(StringIO(r.text), header=None, names=['d_place', 'cnt'])
    df = df[df['d_place'] != target_place]
#     print(df)
    
    min_cnt = min(min_cnt, df['cnt'].min())
    max_cnt = max(max_cnt, df['cnt'].max())
    max_cnt_countries.append(df['cnt'].max())
    dict_countries[target_place] = {}
    dict_countries[target_place]['dataframe'] = df
    dict_countries[target_place]['max_cnt'] = df['cnt'].max()
#     print(target_place, dict_countries[target_place]['max_cnt'])
    


set the start and end points of flows

In [124]:
def set_endpoints(G, dict_countries):
    for target_place in G:        
        df_c = dict_countries[target_place]['dataframe']
        df_c["o_place"] = target_place
        df_c = df_c.set_index("o_place").join(gdf.set_index("GID_0"), how="left")
        df_c = df_c.drop(['NAME_0', "geometry"], axis=1)
        df_c = df_c.rename(columns={"y": "o_lat", 'x': "o_lon"})
        df_c['o_place'] = df_c.index
        df_c = df_c.set_index("d_place").join(gdf.set_index("GID_0"), how="left")
        df_c = df_c.drop(['NAME_0', "geometry"], axis=1)
        df_c = df_c.rename(columns={"y": "d_lat", 'x': "d_lon"})
        df_c['d_place'] = df_c.index
        df_c = df_c[df_c['d_place'] != df_c['o_place']]
        df_c = df_c[~df_c['d_place'].isin(G)]

        dict_countries[target_place]['dataframe'] = df_c
        
set_endpoints(G, dict_countries)

## Show the map

In [146]:
# load the config file for keplergl
kepler_cfg_file = r'kepler_cfg_G.json'
kepler_cfg = json.load(open(kepler_cfg_file, 'r'))


map_2 = KeplerGl(height = 600)
# map_2.config = kepler_cfg

# add data lyaer to map
max_arc_width = 20

for idx in range(len(G)):
#     target_place = G[idx]
    
    dataId = kepler_cfg['config']['visState']['layers'][idx]['config']['dataId']
#     print(dataId)
    map_2.add_data(data=dict_countries[dataId]['dataframe'], name=dataId)  
    
    arc_width = dict_countries[dataId]['max_cnt']/max(max_cnt_countries) * max_arc_width
    arc_width = int(arc_width)
    kepler_cfg['config']['visState']['layers'][idx]['config']['visConfig']['sizeRange'] = [0, arc_width]
    
#     print(kepler_cfg['config']['visState']['layers'][idx]['config']['visConfig']['sizeRange'] )
#     print(kepler_cfg['config']['visState']['layers'][idx]['config']['dataId']  )
#     print(map_2.config['config']['visState']['layers'][idx]['config']['visConfig']['sizeRange'] )
#     print(arc_width)    

# for idx, target_place in enumerate(G):   
#     print(target_place)
#     map_2.add_data(data=dict_counties[target_place], name=target_place)
#     arc_width = max_cnt_countries[idx]/max(max_cnt_countries) * max_arc_width
#     arc_width = int(arc_width)
#     kepler_cfg['config']['visState']['layers'][idx]['config']['visConfig']['sizeRange'] = [0, arc_width]
    
#     print(kepler_cfg['config']['visState']['layers'][idx]['config']['visConfig']['sizeRange'] )
#     print(kepler_cfg['config']['visState']['layers'][idx]['config']['dataId']  )
# #     print(map_2.config['config']['visState']['layers'][idx]['config']['visConfig']['sizeRange'] )
#     print(arc_width)

# show the map
map_2.config = kepler_cfg
map_2

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [{'id': '7bpj7x6', 'type': …

If needed, save the config

In [144]:
cfg = map_2.config
with open('kepler_cfg_G.json', 'w') as fp:
    json.dump(cfg, fp,  indent=2)